In [ ]:
import numpy as np
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
import datetime
import time

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
desti1 = pd.read_csv("../input/destinations.csv")

In [ ]:
desti1.info()

In [ ]:
X = desti1.ix[:,1:150]

In [ ]:
X.info()

In [ ]:
y = desti1.ix[:,0:1]

In [ ]:
y.info()

In [ ]:
pca = PCA(n_components=5)
X_r = pca.fit(X).transform(X)

In [ ]:
X_r1 = pd.DataFrame(X_r)

In [ ]:
X_r1["srch_destination_id"] = desti1["srch_destination_id"]

In [ ]:
print('explained variance ratio (first 5 components): %s' % str(pca.explained_variance_ratio_))

In [ ]:
X_r1.info()

We compresses the 149 columns in destinations down to 5 columns, and creates a new DataFrame called X_r1, preserve most of the variance in destinations, to save a lot of runtime for a machine learning algorithm.

In [ ]:

X_r1.columns = ['d1','d2','d3','d4','d5','srch_destination_id']

In [ ]:
train1 = pd.read_csv("../input/train.csv", nrows=1000000)

In [ ]:
train1["srch_ci"] = pd.to_datetime(train1["srch_ci"], format='%Y-%m-%d', errors="coerce")

In [ ]:
test1["srch_ci"] = pd.to_datetime(test1["srch_ci"], format='%Y-%m-%d', errors="coerce")

In [ ]:
train1["srch_co"] = pd.to_datetime(train1["srch_co"], format='%Y-%m-%d', errors="coerce")

In [ ]:
test1["srch_co"] = pd.to_datetime(test1["srch_co"], format='%Y-%m-%d', errors="coerce")

In [ ]:
train1["stay_span"] = (train1["srch_co"] - train1["srch_ci"]).astype('timedelta64[D]')

In [ ]:
test1["stay_span"] = (test1["srch_co"] - test1["srch_ci"]).astype('timedelta64[D]')

In [ ]:
train2 = pd.to_datetime(train1["date_time"])

In [ ]:
test2 = pd.to_datetime(test1["date_time"])

In [ ]:
train2 = pd.DataFrame(train2)

In [ ]:
test2 = pd.DataFrame(test2)

train1['year'] = train2['date_time'].dt.year
train1['month'] = train2['date_time'].dt.month
train1['day_of_week'] = train2['date_time'].dt.dayofweek
train1['day'] = train2['date_time'].dt.day
train1['hour'] = train2['date_time'].dt.hour

In [ ]:
test1['year'] = test2['date_time'].dt.year
test1['month'] = test2['date_time'].dt.month
test1['day_of_week'] = test2['date_time'].dt.dayofweek
test1['day'] = test2['date_time'].dt.day
test1['hour'] = test2['date_time'].dt.hour

In [ ]:
train1["date_time"] = pd.to_datetime(train1["date_time"], format='%Y-%m-%d', errors="coerce")
train1["srch_ci"] = pd.to_datetime(train1["srch_ci"], format='%Y-%m-%d', errors="coerce")
train1["search_span"] = (train1["srch_ci"] - train1["date_time"]).astype('timedelta64[D]')

In [ ]:
test1["date_time"] = pd.to_datetime(test1["date_time"], format='%Y-%m-%d', errors="coerce")
test1["srch_ci"] = pd.to_datetime(test1["srch_ci"], format='%Y-%m-%d', errors="coerce")
test1["search_span"] = (test1["srch_ci"] - test1["date_time"]).astype('timedelta64[D]')

In [ ]:
train1.ix[(train1['hour'] >= 10) & (train1['hour'] < 18), 'hour'] = 1
train1.ix[(train1['hour'] >= 18) & (train1['hour'] < 22), 'hour'] = 2
train1.ix[(train1['hour'] >= 22) & (train1['hour'] == 24), 'hour'] = 3
train1.ix[(train1['hour'] >= 1) & (train1['hour'] < 10), 'hour'] = 3

In [ ]:
test1.ix[(test1['hour'] >= 10) & (test1['hour'] < 18), 'hour'] = 1
test1.ix[(test1['hour'] >= 18) & (test1['hour'] < 22), 'hour'] = 2
test1.ix[(test1['hour'] >= 22) & (test1['hour'] == 24), 'hour'] = 3
test1.ix[(test1['hour'] >= 1) & (test1['hour'] < 10), 'hour'] = 3

In [ ]:
train1 = train1.drop('srch_ci', axis=1)

In [ ]:
test1 = test1.drop('srch_ci', axis=1)

In [ ]:
train1 = train1.drop('srch_co', axis=1)

In [ ]:
test1 = test1.drop('srch_co', axis=1)

In [ ]:
train1 = train1.drop('date_time', axis=1)

In [ ]:
test1 = test1.drop('date_time', axis=1)

In [ ]:
train1 = train1.join(X_r1, on = 'srch_destination_id', how = 'left', rsuffix='dest')

In [ ]:
train1 = train1.drop("srch_destination_iddest", axis=1)

In [ ]:
train1.fillna(-1, inplace=True)

In [ ]:
test1.fillna(-1, inplace=True)

In [ ]:
train1.info()

In [ ]:
hotelCluster = train1.ix[:,'hotel_cluster']

In [ ]:
hotelCluster1 = pd.DataFrame(hotelCluster)

In [ ]:
hotelCluster1.info()

In [ ]:
hotelCluster1['hotel_cluster'].head()

In [ ]:
train1 = train1.drop('hotel_cluster', axis=1) #df.drop('reports', axis=1)

train1.info()

In [ ]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
#clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)

In [ ]:
clf = MLPClassifier(algorithm='adam', alpha=1e-5, hidden_layer_sizes=(100, 3), learning_rate='adaptive', random_state=1)

In [ ]:
print(hotelCluster.shape)

In [ ]:
clf.fit(train1, hotelCluster)

In [ ]:
test1 = pd.read_csv("../input/test.csv", parse_dates=['date_time'], nrows=10)

In [ ]:
test1.info()

In [ ]:
test1['j'] = -1

clf.predict_proba(test1)